In [ ]:
import os
import zipfile
import shutil
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import numpy as np

##Read Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
zip_file_path = '/content/drive/MyDrive/archive (1).zip'
print("File exists?" , os.path.exists(zip_file_path))

File exists? True


In [ ]:
zip_file_path = '/content/drive/MyDrive/archive (1).zip'

In [ ]:
print(os.path.exists('/content/drive/MyDrive/archive (1).zip'))

True


In [ ]:
extract_folder = '/content/dataset'


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("Extracted files:", os.listdir(extract_folder))


Extracted files: ['COVID-19_Radiography_Dataset', 'metadata.csv']


In [ ]:
print(os.listdir('/content/dataset/COVID-19_Radiography_Dataset'))


['COVID-19_Radiography_Dataset']


In [ ]:
print(os.listdir('/content/dataset/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset'))


['Viral Pneumonia', 'COVID', 'Normal', 'Lung_Opacity']


##Images Preprocessing

In [ ]:

dataset_dir = '/content/dataset/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset'


classes = ['Viral Pneumonia', 'COVID', 'Normal', 'Lung_Opacity']
class_mapping = {'Viral Pneumonia': 'pneumonia', 'COVID': 'covid19', 'Normal': 'normal', 'Lung_Opacity': 'opacity'}


train_dir = '/content/dataset/train'
val_dir = '/content/dataset/val'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)


for class_name in class_mapping.values():
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)


for class_folder in classes:
    class_path = os.path.join(dataset_dir, class_folder)
    class_name = class_mapping[class_folder]


    images = os.listdir(class_path)


    random.shuffle(images)


    split_index = int(0.8 * len(images))


    train_images = images[:split_index]

    val_images = images[split_index:]


    for image in train_images:
        src = os.path.join(class_path, image)
        dst = os.path.join(train_dir, class_name, image)
        shutil.copy(src, dst)

    for image in val_images:
        src = os.path.join(class_path, image)
        dst = os.path.join(val_dir, class_name, image)
        shutil.copy(src, dst)

print("Data split successfully.")


Data split successfully.


##Oversampling

In [ ]:

src_dir = '/content/dataset/train/pneumonia'
images = os.listdir(src_dir)


current_count = len(images)
target_count = 4000


if current_count < target_count:
    extra_needed = target_count - current_count
    for i in range(extra_needed):
        img_name = random.choice(images)
        src = os.path.join(src_dir, img_name)
        new_name = f"aug_{i}_{img_name}"
        dst = os.path.join(src_dir, new_name)
        shutil.copy(src, dst)

print(f"oversampling done and add {extra_needed} images")


oversampling done and add 2924 images


##Class Weights

In [ ]:

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
)


val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


train_dir = '/content/dataset/train'
val_dir = '/content/dataset/val'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)


val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

labels = train_generator.classes
class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights = {i: np.float64(class_weights_array[i]) for i in range(len(class_weights_array))}
print("Class Weights:", class_weights)



Found 19854 images belonging to 4 classes.
Found 4235 images belonging to 4 classes.
Class Weights: {0: np.float64(1.7162863070539418), 1: np.float64(0.6087943088433705), 2: np.float64(1.0321272613849033), 3: np.float64(1.240875)}


##Model Building

In [ ]:

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))


base_model.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers[-50:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 71, 71,    │          0 │ batch_normalizat

 Total params: 22,065,572 (84.17 MB)

 Trainable params: 7,436,100 (28.37 MB)

 Non-trainable params: 14,629,472 (55.81 MB)

##Callbacks

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/best_model_inception.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_accuracy', patience=3, factor=0.5, min_lr=1e-7, verbose=1)
early = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

callbacks_list = [checkpoint, early, lr_schedule]


##Model Training

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    class_weight=class_weights,
    callbacks=callbacks_list
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 752ms/step - accuracy: 0.7766 - loss: 0.5405
Epoch 1: val_accuracy improved from -inf to 0.89988, saving model to /content/drive/MyDrive/best_model_inception.h5


621/621 ━━━━━━━━━━━━━━━━━━━━ 533s 810ms/step - accuracy: 0.7767 - loss: 0.5403 - val_accuracy: 0.8999 - val_loss: 0.2705 - learning_rate: 1.0000e-04
Epoch 2/10
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 733ms/step - accuracy: 0.8961 - loss: 0.2619
Epoch 2: val_accuracy improved from 0.89988 to 0.91287, saving model to /content/drive/MyDrive/best_model_inception.h5


621/621 ━━━━━━━━━━━━━━━━━━━━ 480s 773ms/step - accuracy: 0.8961 - loss: 0.2619 - val_accuracy: 0.9129 - val_loss: 0.2426 - learning_rate: 1.0000e-04
Epoch 3/10
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 736ms/step - accuracy: 0.9171 - loss: 0.2018
Epoch 3: val_accuracy improved from 0.91287 to 0.92774, saving model to /content/drive/MyDrive/best_model_inception.h5


621/621 ━━━━━━━━━━━━━━━━━━━━ 481s 774ms/step - accuracy: 0.9171 - loss: 0.2018 - val_accuracy: 0.9277 - val_loss: 0.2054 - learning_rate: 1.0000e-04
Epoch 4/10
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 736ms/step - accuracy: 0.9277 - loss: 0.1694
Epoch 4: val_accuracy improved from 0.92774 to 0.92845, saving model to /content/drive/MyDrive/best_model_inception.h5


621/621 ━━━━━━━━━━━━━━━━━━━━ 483s 778ms/step - accuracy: 0.9277 - loss: 0.1694 - val_accuracy: 0.9285 - val_loss: 0.2107 - learning_rate: 1.0000e-04
Epoch 5/10
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 722ms/step - accuracy: 0.9325 - loss: 0.1605
Epoch 5: val_accuracy did not improve from 0.92845
621/621 ━━━━━━━━━━━━━━━━━━━━ 488s 756ms/step - accuracy: 0.9325 - loss: 0.1605 - val_accuracy: 0.9285 - val_loss: 0.2090 - learning_rate: 1.0000e-04
Epoch 6/10
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 732ms/step - accuracy: 0.9401 - loss: 0.1461
Epoch 6: val_accuracy did not improve from 0.92845
621/621 ━━━━━━━━━━━━━━━━━━━━ 475s 765ms/step - accuracy: 0.9401 - loss: 0.1461 - val_accuracy: 0.9044 - val_loss: 0.2717 - learning_rate: 1.0000e-04
Epoch 7/10
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 737ms/step - accuracy: 0.9440 - loss: 0.1378
Epoch 7: val_accuracy did not improve from 0.92845

Epoch 7: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
621/621 ━━━━━━━━━━━━━━━━━━━━ 479s 771ms/step - accuracy: 0.

621/621 ━━━━━━━━━━━━━━━━━━━━ 481s 774ms/step - accuracy: 0.9495 - loss: 0.1159 - val_accuracy: 0.9327 - val_loss: 0.2004 - learning_rate: 5.0000e-05
Epoch 9/10
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 735ms/step - accuracy: 0.9581 - loss: 0.1018
Epoch 9: val_accuracy did not improve from 0.93270
621/621 ━━━━━━━━━━━━━━━━━━━━ 477s 769ms/step - accuracy: 0.9581 - loss: 0.1018 - val_accuracy: 0.9287 - val_loss: 0.2111 - learning_rate: 5.0000e-05
Epoch 10/10
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 736ms/step - accuracy: 0.9587 - loss: 0.0964
Epoch 10: val_accuracy did not improve from 0.93270
621/621 ━━━━━━━━━━━━━━━━━━━━ 478s 769ms/step - accuracy: 0.9587 - loss: 0.0964 - val_accuracy: 0.9308 - val_loss: 0.2238 - learning_rate: 5.0000e-05
Restoring model weights from the end of the best epoch: 8.


In [ ]:

model.save('/content/drive/MyDrive/best_model_inception.h5')


In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/best_model_inception.h5', compile=False)


In [ ]:
for layer in model.layers:
    layer.trainable = False


for layer in model.layers[-70:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.fit(
    train_generator,
    epochs=15,
    initial_epoch=10,
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=callbacks_list
)

Epoch 11/15
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 744ms/step - accuracy: 0.9517 - loss: 0.1130
Epoch 11: val_accuracy did not improve from 0.93270
621/621 ━━━━━━━━━━━━━━━━━━━━ 514s 790ms/step - accuracy: 0.9517 - loss: 0.1130 - val_accuracy: 0.9327 - val_loss: 0.1987 - learning_rate: 1.0000e-05
Epoch 12/15
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 728ms/step - accuracy: 0.9539 - loss: 0.1053
Epoch 12: val_accuracy improved from 0.93270 to 0.93459, saving model to /content/drive/MyDrive/best_model_inception.h5


621/621 ━━━━━━━━━━━━━━━━━━━━ 476s 767ms/step - accuracy: 0.9540 - loss: 0.1053 - val_accuracy: 0.9346 - val_loss: 0.1920 - learning_rate: 1.0000e-05
Epoch 13/15
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 732ms/step - accuracy: 0.9542 - loss: 0.1052
Epoch 13: val_accuracy did not improve from 0.93459
621/621 ━━━━━━━━━━━━━━━━━━━━ 475s 765ms/step - accuracy: 0.9542 - loss: 0.1052 - val_accuracy: 0.9332 - val_loss: 0.1986 - learning_rate: 1.0000e-05
Epoch 14/15
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 730ms/step - accuracy: 0.9612 - loss: 0.0888
Epoch 14: val_accuracy improved from 0.93459 to 0.93577, saving model to /content/drive/MyDrive/best_model_inception.h5


621/621 ━━━━━━━━━━━━━━━━━━━━ 477s 768ms/step - accuracy: 0.9612 - loss: 0.0888 - val_accuracy: 0.9358 - val_loss: 0.1983 - learning_rate: 1.0000e-05
Epoch 15/15
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 738ms/step - accuracy: 0.9617 - loss: 0.0908
Epoch 15: val_accuracy did not improve from 0.93577
621/621 ━━━━━━━━━━━━━━━━━━━━ 479s 771ms/step - accuracy: 0.9617 - loss: 0.0908 - val_accuracy: 0.9313 - val_loss: 0.2030 - learning_rate: 1.0000e-05
Restoring model weights from the end of the best epoch: 12.


In [ ]:
model.save('/content/drive/MyDrive/best_model_inception.h5')


In [ ]:
model = load_model('/content/drive/MyDrive/best_model_inception.h5', compile=False)

In [ ]:
for layer in model.layers:
    layer.trainable = False

for layer in model.layers[-100:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.fit(
    train_generator,
    epochs=20,
    initial_epoch=15,
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=callbacks_list
)

Epoch 16/20
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 770ms/step - accuracy: 0.9546 - loss: 0.1085
Epoch 16: val_accuracy improved from 0.93577 to 0.93601, saving model to /content/drive/MyDrive/best_model_inception.h5


621/621 ━━━━━━━━━━━━━━━━━━━━ 540s 821ms/step - accuracy: 0.9546 - loss: 0.1085 - val_accuracy: 0.9360 - val_loss: 0.1908 - learning_rate: 1.0000e-05
Epoch 17/20
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 753ms/step - accuracy: 0.9571 - loss: 0.1029
Epoch 17: val_accuracy did not improve from 0.93601
621/621 ━━━━━━━━━━━━━━━━━━━━ 489s 787ms/step - accuracy: 0.9571 - loss: 0.1029 - val_accuracy: 0.9353 - val_loss: 0.1888 - learning_rate: 1.0000e-05
Epoch 18/20
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 740ms/step - accuracy: 0.9592 - loss: 0.0924
Epoch 18: val_accuracy improved from 0.93601 to 0.93932, saving model to /content/drive/MyDrive/best_model_inception.h5


621/621 ━━━━━━━━━━━━━━━━━━━━ 497s 780ms/step - accuracy: 0.9592 - loss: 0.0924 - val_accuracy: 0.9393 - val_loss: 0.1862 - learning_rate: 1.0000e-05
Epoch 19/20
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 774ms/step - accuracy: 0.9603 - loss: 0.0926
Epoch 19: val_accuracy did not improve from 0.93932
621/621 ━━━━━━━━━━━━━━━━━━━━ 502s 808ms/step - accuracy: 0.9603 - loss: 0.0926 - val_accuracy: 0.9386 - val_loss: 0.1823 - learning_rate: 1.0000e-05
Epoch 20/20
621/621 ━━━━━━━━━━━━━━━━━━━━ 0s 759ms/step - accuracy: 0.9621 - loss: 0.0872
Epoch 20: val_accuracy did not improve from 0.93932
621/621 ━━━━━━━━━━━━━━━━━━━━ 493s 794ms/step - accuracy: 0.9621 - loss: 0.0872 - val_accuracy: 0.9381 - val_loss: 0.1878 - learning_rate: 1.0000e-05
Restoring model weights from the end of the best epoch: 19.


In [ ]:
model.save('/content/drive/MyDrive/best_model_inception.h5')


In [ ]:
from google.colab import files


files.download('/content/drive/MyDrive/best_model_inception.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>